In [1]:
import numpy as np
import pandas as pd
try:
    import openmm as mm
    import openmm.app as app
    import openmm.unit as unit
except ImportError:
    import simtk.openmm as mm
    import simtk.openmm.app as app
    import simtk.unit as unit
import mdtraj
import sys
import os

try:
    import nglview
except ImportError:
    print('Please install nglview to visualize molecules in the jupyter notebooks.')

sys.path.append('../..')
from openabc.forcefields.parsers import MpipiProteinParser, MpipiRNAParser
from openabc.forcefields.mpipi_model import MpipiModel
import openabc.utils.helper_functions as helper_functions
from openabc.utils.insert import insert_molecules

# set simulation platform
#platform_name = 'CPU'
platform_name = 'CUDA'
#platform_name = 'OpenCL'


Here we show how to use Mpipi model to set up protein-RNA simulations. We use a mixture of polyR, polyK, and polyU as an example. First we build individual CG chains with given sequence, then we insert chains into a simulation box. Finally we run the MD simulation. 

In [2]:
# build residue level CG atom chains
sequence = 'GSMASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGSYNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF' # WT
#sequence = 'GSMASASSSQRGRSGSGNSGGGRGGGFGGNDNFGRGGNSSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNSGGGGSSNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYSAKPRNQGGYGGSSSSSSSGSGRRF' # ARO-
ca_pdb = 'init_A1LCD_WT_CA.pdb'
ca_atoms = helper_functions.build_straight_CA_chain(sequence) # r0=0.38
helper_functions.write_pdb(ca_atoms, ca_pdb)

# parse individual protein and RNA
a1lcd = MpipiProteinParser(ca_pdb)
#polyU = MpipiRNAParser('polyU_CG.pdb')

Parse molecule with default settings.


In [3]:
# insert chains into a box
n_mol = 50
box_size = 100
if not os.path.exists('start.pdb'):
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[box_size, box_size, box_size])
else:
    # delete and create a new start.pdb
    # otherwise, there will be an error for init_coord in the next step
    os.remove('start.pdb')
    insert_molecules(ca_pdb, 'start.pdb', n_mol, box=[box_size, box_size, box_size])

Successfully inserted 50 molecules.


In [4]:
# visualize start.pdb
start_pdb = mdtraj.load_pdb('start.pdb')
view = nglview.show_mdtraj(start_pdb)
view

NGLWidget()

In [5]:
# set up simulation
top = app.PDBFile('start.pdb').getTopology()
a1lcd_condensate = MpipiModel()
for i in range(n_mol):
    a1lcd_condensate.append_mol(a1lcd)
a1lcd_condensate.create_system(top, box_a=box_size, box_b=box_size, box_c=box_size)
a1lcd_condensate.add_protein_bonds(force_group=1)
#a1lcd_condensate.add_rna_bonds(force_group=2)
a1lcd_condensate.add_contacts(force_group=3)
a1lcd_condensate.add_dh_elec(ldby=(1/1.26)*unit.nanometer, force_group=4)
temperature = 300*unit.kelvin
friction_coeff = 1/unit.picosecond
timestep = 10*unit.femtosecond
integrator = mm.LangevinMiddleIntegrator(temperature, friction_coeff, timestep)
init_coord = app.PDBFile('start.pdb').getPositions()
a1lcd_condensate.set_simulation(integrator, platform_name='CPU', init_coord=init_coord)
a1lcd_condensate.simulation.minimizeEnergy()
output_interval = 1000
output_dcd = 'output.dcd'
a1lcd_condensate.add_reporters(output_interval, output_dcd)
a1lcd_condensate.simulation.context.setVelocitiesToTemperature(temperature)
a1lcd_condensate.simulation.step(10000)

Add protein bonds.
Add nonbonded contacts.
Add Debye-Huckel electrostatic interactions.
Set Debye length as 0.7936507936507936 nm.
Set water dielectric as 80.0.
Use platform: CPU
#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,9.999999999999831,11374.082802813911,25594.566739859503,36968.64954267342,299.63681492199635,0
2000,20.000000000000327,9667.28862860752,26171.02218925213,35838.31081785965,306.38540639283536,139
3000,30.00000000000189,9214.256684824151,25727.416121493323,34941.672806317474,301.19208897612293,137
4000,40.00000000000061,8808.625514830886,25496.286965463452,34304.91248029434,298.4862489104442,135
5000,49.99999999999862,8019.570466668446,25488.011973351982,33507.58244002043,298.389373182678,134
6000,59.99999999999663,7453.66057297511,25603.965343567084,33057.625916542194,299.74684482436885,137
7000,69.9999999999989,7091.843449108792,25458.43924020947,32550.282689318265,298.04

In [6]:
# view trajectory
traj = mdtraj.load_dcd('output.dcd', top='start.pdb')
traj.xyz -= np.mean(traj.xyz, axis=1, keepdims=True) # realign to the origin
view = nglview.show_mdtraj(traj)
view

NGLWidget(max_frame=9)